In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('/content/drive/MyDrive/data/fake news classifier/train.csv/train.csv')

In [3]:
df  = df.dropna()

In [4]:
X = df.drop('label',axis=1)
y = df['label']

In [5]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [7]:
voc_size = 10000
# one hot representation


In [8]:
message = X.copy()
message.reset_index(inplace =True)

In [9]:
message

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [10]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
 message['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [12]:
## Data preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(message)):
  review = re.sub('[^a-zA-Z]',' ',message['title'][i])
  review = review.lower()
  review  = review.split()
  review = [ps.stem(word)   for word in review   if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [13]:
one_hot_repr = [one_hot(word,voc_size)  for word in corpus]

In [14]:
one_hot_repr

[[5303, 4274, 7485, 7865, 5991, 7018, 9974, 1361, 1800, 5718],
 [3969, 7116, 8346, 8031, 2204, 4507, 7837],
 [8243, 3808, 6982, 4505],
 [1314, 4526, 2306, 8034, 8954, 8538],
 [3172, 2204, 8877, 3015, 5824, 7689, 2204, 1614, 4456, 2414],
 [211,
  3814,
  6080,
  3372,
  4812,
  4994,
  5663,
  6341,
  2506,
  4643,
  540,
  7559,
  6281,
  7740,
  7837],
 [9272, 3014, 6976, 7698, 2747, 6894, 3163, 5112, 1422, 5698, 6406],
 [7901, 4911, 5774, 7844, 4897, 7417, 4994, 3613, 1422, 5698, 6406],
 [5609, 3093, 3104, 8525, 2338, 8684, 4058, 5151, 4994, 3672],
 [8829, 6271, 6624, 577, 6189, 957, 2094, 6744],
 [2155, 7883, 4359, 4870, 4807, 594, 966, 5266, 5698, 2345, 3669],
 [8034, 215, 5991, 8684, 4994, 4897],
 [3244, 9494, 143, 6381, 4691, 9838, 1226, 8206, 8656],
 [593, 4107, 96, 5241, 9666, 6010, 3883, 1422, 5698, 6406],
 [2765, 9772, 9538, 7753, 78, 1422, 5698, 6406],
 [1497, 4544, 4498, 8171, 7899, 434, 1295, 1767, 2972, 3055],
 [2737, 7988, 7116],
 [8902, 9224, 331, 723, 4994, 3358, 7323,

In [15]:
## embedding representation
sent_length = 20
embedded_doc = pad_sequences(one_hot_repr,padding  = 'pre',maxlen = sent_length)

In [16]:
# creating model
embedding_vector_feature = 40
model  =Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 513,001
Trainable params: 513,001
Non-trainable params: 0
_________________________________________________________________


In [18]:
X_final = np.array(embedded_doc)
y_final  = np.array(y)

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size = 0.33,random_state = 42)

In [20]:
## model training
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 100,batch_size = 64)

Epoch 1/100
192/192 [==============================] - 12s 15ms/step - loss: 0.2949 - accuracy: 0.8651 - val_loss: 0.1934 - val_accuracy: 0.9191
Epoch 2/100
192/192 [==============================] - 2s 12ms/step - loss: 0.1250 - accuracy: 0.9562 - val_loss: 0.1955 - val_accuracy: 0.9223
Epoch 3/100
192/192 [==============================] - 2s 11ms/step - loss: 0.0755 - accuracy: 0.9744 - val_loss: 0.2338 - val_accuracy: 0.9168
Epoch 4/100
192/192 [==============================] - 2s 11ms/step - loss: 0.0508 - accuracy: 0.9837 - val_loss: 0.2330 - val_accuracy: 0.9137
Epoch 5/100
192/192 [==============================] - 2s 11ms/step - loss: 0.0306 - accuracy: 0.9909 - val_loss: 0.3288 - val_accuracy: 0.9175
Epoch 6/100
192/192 [==============================] - 2s 11ms/step - loss: 0.0197 - accuracy: 0.9947 - val_loss: 0.3937 - val_accuracy: 0.9085
Epoch 7/100
192/192 [==============================] - 2s 11ms/step - loss: 0.0147 - accuracy: 0.9963 - val_loss: 0.4255 - val_accuracy

In [22]:
#performance matrix
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [23]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]], dtype=int32)

In [24]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
acc_scr  = accuracy_score(y_test,y_pred)



In [25]:
cm,acc_scr

(array([[3115,  304],
        [ 272, 2344]]), 0.9045567522783762)

In [27]:
from sklearn.metrics import classification_report
classification_report(y_test,y_pred)

'              precision    recall  f1-score   support\n\n           0       0.92      0.91      0.92      3419\n           1       0.89      0.90      0.89      2616\n\n    accuracy                           0.90      6035\n   macro avg       0.90      0.90      0.90      6035\nweighted avg       0.90      0.90      0.90      6035\n'